# Detect Cars Tutorial
This notebook shows how to use every vision capability in Moondream.
You'll caption an image, ask a question about it, detect cars with boxes,
point to each car's center, and then process an entire folder, saving the
overlayed images and a JSON summary.

In [ ]:
import os
import json
from PIL import Image, ImageDraw
from IPython.display import display
import moondream as md

# Initialize the model to run locally
model = md.vl(endpoint="http://localhost:2020/v1")
# For Moondream Cloud use:
# model = md.vl(api_key="<your-api-key>")

INPUT_FOLDER = os.path.join(os.path.dirname(__file__), '..', '..', 'images')
OUTPUT_FOLDER = os.path.join(INPUT_FOLDER, 'detected')
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

image_files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(('png','jpg','jpeg'))]

## Run all capabilities on one image
We'll caption the first image, ask a question, detect cars with boxes, and point to them.

In [ ]:
first_path = os.path.join(INPUT_FOLDER, image_files[0])
first_image = Image.open(first_path)
caption = model.caption(first_image)['caption']
answer = model.query(first_image, "What's in this image?")['answer']
detection = model.detect(first_image, 'car')['objects']
points = model.point(first_image, 'car')['points']
overlay = first_image.copy()
draw = ImageDraw.Draw(overlay)
w, h = overlay.size
for box in detection:
    draw.rectangle([
        int(box['x_min'] * w),
        int(box['y_min'] * h),
        int(box['x_max'] * w),
        int(box['y_max'] * h)
    ], outline='red', width=3)
for pt in points:
    r = 4
    draw.ellipse([
        int(pt['x'] * w) - r, int(pt['y'] * h) - r,
        int(pt['x'] * w) + r, int(pt['y'] * h) + r
    ], fill='blue')
display(overlay)
caption, answer, detection, points

## Process the entire folder
Now let's run caption, query, detect, and point on every image, saving overlayed versions and recording the results in a JSON file.

In [ ]:
results = []
for fname in image_files:
    img_path = os.path.join(INPUT_FOLDER, fname)
    img = Image.open(img_path)
    caption = model.caption(img)['caption']
    answer = model.query(img, "What's in this image?")['answer']
    boxes = model.detect(img, 'car')['objects']
    points = model.point(img, 'car')['points']
    ov = img.copy()
    d = ImageDraw.Draw(ov)
    w, h = ov.size
    for b in boxes:
        d.rectangle([
            int(b['x_min'] * w),
            int(b['y_min'] * h),
            int(b['x_max'] * w),
            int(b['y_max'] * h)
        ], outline='red', width=3)
    for p in points:
        r = 4
        d.ellipse([
            int(p['x'] * w) - r, int(p['y'] * h) - r,
            int(p['x'] * w) + r, int(p['y'] * h) + r
        ], fill='blue')
    out_path = os.path.join(OUTPUT_FOLDER, fname)
    ov.save(out_path)
    results.append({'original': img_path, 'overlay': out_path, 'caption': caption, 'answer': answer, 'boxes': boxes, 'points': points})

json_path = os.path.join(OUTPUT_FOLDER, 'results.json')
with open(json_path, 'w') as f:
    json.dump(results, f, indent=2)

f'Saved {len(results)} results to {json_path}'